We'll then import these packages and declare the function that retrieves post titles from reddit.

In [14]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)
import nltk

df  = pd.read_excel('../Data/Example_data.xlsx')
#df['Description'].head(3)
df.columns

Index(['Domain', 'Title', 'Description', 'Body', 'Link', 'timestamp',
       'Analyst_Average_Score', 'Analyst_Rank', 'Reference_Final_Score'],
      dtype='object')

# After a text is obtained, we start with text normalization. Text normalization includes:

* converting all letters to lower or upper case 
* converting numbers into words or removing numbers 
* removing punctuations, accent marks and other diacritics 
* removing white spaces 
* expanding abbreviations 
* removing stop words, sparse terms, and particular words 
* text canonicalization 

# Step 1: Tokenization

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mohammed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]

    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()] 

NameError: name 'DataFrame' is not defined

In [21]:
df['tokenized_Description'] = df.apply(lambda x: tokenize(x['Description']), axis=1)
df[['Description', 'tokenized_Description']].head()

,Description,tokenized_Description
0,…often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…,"[trigger, a, protest, vote, that, can, ministers, who, breach, the, Ministerial, up, the, in, order, questions, about, abuse, of, taxpayers, money, which, is, a, clear, breach, of, the, not, permit, abuse, of, taxpayers]"
1,"Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.","[Stumbled, across, an, interesting, case, a, woman, facing, eviction, from, the, home, that, she, inherited, from, her, brother, in, Turns, out, her, brother, was, in, business, with, a, notorious, international, criminal, Paul, Le, Roux, who, is, currently, serving, years, in, a, New, York, prison]"
2,…COVID-19…COVID…COVID…COVID-19 et Post COVID…COVID…Covid…BASF…COVID…COVID-19 3.4.1 Évaluation des risques liés au COVID-19 3.4.2 Évaluation de l’impact global du COVID-19 sur l’industrie 3.4.3 Scénario de marché avant et après COVID…covid…covid…,"[et, Post, Évaluation, des, risques, liés, au, Évaluation, de, l, impact, global, du, sur, l, industrie, Scénario, de, marché, avant, et, après]"
3,…hate raiders' linked to automated harassment campaigns [engadget](d.php?did=engadget.com) Texas law could force social media to host misinformation and hate speech [ars](d.php?did=arstechnica.com)…breach…shot on…stolen data…Killed…M1…Toxic…Attack…,"[raiders, linked, to, automated, harassment, campaigns, engadget, Texas, law, could, force, social, media, to, host, misinformation, and, hate, speech, ars]"
4,"…Abschnitten und Endanwendungen / Organisationen. Covid-19 kann die Weltwirtschaft auf…/ Key-Player profiliert: Eastman Polynt Lanxess BASF Daicel Jiangsu Ruijia Jiangsu Lemon…Japan, wie Eastman, Lanxess, BASF, Daicel und Jiangsu Ruijia konzentriert.…","[und, Endanwendungen, Organisationen, kann, die, Weltwirtschaft, profiliert, Eastman, Polynt, Lanxess, BASF, Daicel, Jiangsu, Ruijia, Jiangsu, wie, Eastman, Lanxess, BASF, Daicel, und, Jiangsu, Ruijia]"


In [22]:
df['tokenized_Title'] = df.apply(lambda x: tokenize(x['Title']), axis=1)
df[['Title', 'tokenized_Title']].head()

,Title,tokenized_Title
0,Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for,"[Boris, Johnson, using, a, jet, for, an, election, campaign, fits, a, long, history, of, taking, things, he, did, pay, for]"
1,"Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.","[Stumbled, across, an, interesting, case, a, woman, facing, eviction, from, the, home, that, she, inherited, from, her, brother, in, Turns, out, her, brother, was, in, business, with, a, notorious, international, criminal, Paul, Le, Roux, who, is, currently, serving, years, in, a, New, York, prison]"
2,"Marché Résines dans les peintures et revêtements 2021 avec les données des meilleurs pays et l’analyse Covid-19, la portée future, l’estimation de la taille, les revenus, les tendances des prix et les prévisions d’ici 2026","[Marché, Résines, dans, les, peintures, et, revêtements, avec, les, données, des, meilleurs, pays, et, l, analyse, la, portée, future, l, estimation, de, la, taille, les, revenus, les, tendances, des, prix, et, les, prévisions, d, ici]"
3,"AI drives data analytics surge, study finds","[AI, drives, data, analytics, surge, study, finds]"
4,"Triacetin Vertrieb Markt 2021: Globale Unternehmensanalyse, Merkmale, Marktplatzlänge und -prognosen bis zu 2027 mit Hauptregionen und Nationeninformationen","[Triacetin, Vertrieb, Markt, Globale, Unternehmensanalyse, Merkmale, Marktplatzlänge, und, bis, zu, mit, Hauptregionen, und, Nationeninformationen]"


# Step 2: Stopword removal

In [31]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mohammed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
def remove_stopwords(tokenized_column):
    """Return a list of tokens with English stopwords removed. 

    Args:
        column: Pandas dataframe column of tokenized data from tokenize()

    Returns:
        tokens (list): Tokenized list with stopwords removed.

    """
    stops = set(stopwords.words("english"))
    return [word for word in tokenized_column if not word in stops]

In [33]:
df['DS_stopwords_removed'] = df.apply(lambda x: remove_stopwords(x['tokenized_Description']), axis=1)
df[['Description', 'DS_stopwords_removed']].head()

,Description,DS_stopwords_removed
0,…often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…,"[trigger, protest, vote, ministers, breach, Ministerial, order, questions, abuse, taxpayers, money, clear, breach, permit, abuse, taxpayers]"
1,"Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.","[Stumbled, across, interesting, case, woman, facing, eviction, home, inherited, brother, Turns, brother, business, notorious, international, criminal, Paul, Le, Roux, currently, serving, years, New, York, prison]"
2,…COVID-19…COVID…COVID…COVID-19 et Post COVID…COVID…Covid…BASF…COVID…COVID-19 3.4.1 Évaluation des risques liés au COVID-19 3.4.2 Évaluation de l’impact global du COVID-19 sur l’industrie 3.4.3 Scénario de marché avant et après COVID…covid…covid…,"[et, Post, Évaluation, des, risques, liés, au, Évaluation, de, l, impact, global, du, sur, l, industrie, Scénario, de, marché, avant, et, après]"
3,…hate raiders' linked to automated harassment campaigns [engadget](d.php?did=engadget.com) Texas law could force social media to host misinformation and hate speech [ars](d.php?did=arstechnica.com)…breach…shot on…stolen data…Killed…M1…Toxic…Attack…,"[raiders, linked, automated, harassment, campaigns, engadget, Texas, law, could, force, social, media, host, misinformation, hate, speech, ars]"
4,"…Abschnitten und Endanwendungen / Organisationen. Covid-19 kann die Weltwirtschaft auf…/ Key-Player profiliert: Eastman Polynt Lanxess BASF Daicel Jiangsu Ruijia Jiangsu Lemon…Japan, wie Eastman, Lanxess, BASF, Daicel und Jiangsu Ruijia konzentriert.…","[und, Endanwendungen, Organisationen, kann, die, Weltwirtschaft, profiliert, Eastman, Polynt, Lanxess, BASF, Daicel, Jiangsu, Ruijia, Jiangsu, wie, Eastman, Lanxess, BASF, Daicel, und, Jiangsu, Ruijia]"


In [34]:
df['T_stopwords_removed'] = df.apply(lambda x: remove_stopwords(x['tokenized_Title']), axis=1)
df[['Title', 'T_stopwords_removed']].head()

,Title,T_stopwords_removed
0,Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for,"[Boris, Johnson, using, jet, election, campaign, fits, long, history, taking, things, pay]"
1,"Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.","[Stumbled, across, interesting, case, woman, facing, eviction, home, inherited, brother, Turns, brother, business, notorious, international, criminal, Paul, Le, Roux, currently, serving, years, New, York, prison]"
2,"Marché Résines dans les peintures et revêtements 2021 avec les données des meilleurs pays et l’analyse Covid-19, la portée future, l’estimation de la taille, les revenus, les tendances des prix et les prévisions d’ici 2026","[Marché, Résines, dans, les, peintures, et, revêtements, avec, les, données, des, meilleurs, pays, et, l, analyse, la, portée, future, l, estimation, de, la, taille, les, revenus, les, tendances, des, prix, et, les, prévisions, ici]"
3,"AI drives data analytics surge, study finds","[AI, drives, data, analytics, surge, study, finds]"
4,"Triacetin Vertrieb Markt 2021: Globale Unternehmensanalyse, Merkmale, Marktplatzlänge und -prognosen bis zu 2027 mit Hauptregionen und Nationeninformationen","[Triacetin, Vertrieb, Markt, Globale, Unternehmensanalyse, Merkmale, Marktplatzlänge, und, bis, zu, mit, Hauptregionen, und, Nationeninformationen]"


# Step 3: Stemming

In [35]:
from nltk.stem.porter import PorterStemmer
def apply_stemming(tokenized_column):
    """Return a list of tokens with Porter stemming applied.

    Args:
        column: Pandas dataframe column of tokenized data with stopwords removed.

    Returns:
        tokens (list): Tokenized list with words Porter stemmed.

    """

    stemmer = PorterStemmer() 
    return [stemmer.stem(word) for word in tokenized_column]

In [36]:
df['DS_porter_stemmed'] = df.apply(lambda x: apply_stemming(x['DS_stopwords_removed']), axis=1)
df[['DS_porter_stemmed']].head()

,DS_porter_stemmed
0,"[trigger, protest, vote, minist, breach, ministeri, order, question, abus, taxpay, money, clear, breach, permit, abus, taxpay]"
1,"[stumbl, across, interest, case, woman, face, evict, home, inherit, brother, turn, brother, busi, notori, intern, crimin, paul, le, roux, current, serv, year, new, york, prison]"
2,"[et, post, évaluat, de, risqu, lié, au, évaluat, de, l, impact, global, du, sur, l, industri, scénario, de, marché, avant, et, aprè]"
3,"[raider, link, autom, harass, campaign, engadget, texa, law, could, forc, social, media, host, misinform, hate, speech, ar]"
4,"[und, endanwendungen, organisationen, kann, die, weltwirtschaft, profiliert, eastman, polynt, lanxess, basf, daicel, jiangsu, ruijia, jiangsu, wie, eastman, lanxess, basf, daicel, und, jiangsu, ruijia]"


In [38]:
df['T_porter_stemmed'] = df.apply(lambda x: apply_stemming(x['T_stopwords_removed']), axis=1)
df['T_porter_stemmed'].head(2)

0                                                                                                     [bori, johnson, use, jet, elect, campaign, fit, long, histori, take, thing, pay]
1    [stumbl, across, interest, case, woman, face, evict, home, inherit, brother, turn, brother, busi, notori, intern, crimin, paul, le, roux, current, serv, year, new, york, prison]
Name: T_porter_stemmed, dtype: object

In [41]:
def rejoin_words(tokenized_column):
    """Rejoins a tokenized word list into a single string. 
    
    Args:
        tokenized_column (list): Tokenized column of words. 
        
    Returns:
        string: Single string of untokenized words. 
    """
    
    return ( " ".join(tokenized_column))

In [44]:
df['DS_rejoined'] = df.apply(lambda x: rejoin_words(x['DS_porter_stemmed']), axis=1)
df['DS_rejoined'].head()

0                                                                        trigger protest vote minist breach ministeri order question abus taxpay money clear breach permit abus taxpay
1                              stumbl across interest case woman face evict home inherit brother turn brother busi notori intern crimin paul le roux current serv year new york prison
2                                                                        et post évaluat de risqu lié au évaluat de l impact global du sur l industri scénario de marché avant et aprè
3                                                                            raider link autom harass campaign engadget texa law could forc social media host misinform hate speech ar
4    und endanwendungen organisationen kann die weltwirtschaft profiliert eastman polynt lanxess basf daicel jiangsu ruijia jiangsu wie eastman lanxess basf daicel und jiangsu ruijia
Name: DS_rejoined, dtype: object

In [42]:
df['T_rejoined'] = df.apply(lambda x: rejoin_words(x['T_porter_stemmed']), axis=1)
df['T_rejoined'].head(3)

0                                                                                                         bori johnson use jet elect campaign fit long histori take thing pay
1                     stumbl across interest case woman face evict home inherit brother turn brother busi notori intern crimin paul le roux current serv year new york prison
2    marché résine dan le peintur et revêtement avec le donné de meilleur pay et l analys la porté futur l estim de la taill le revenu le tendanc de prix et le prévision ici
Name: T_rejoined, dtype: object

In [45]:
df.columns

Index(['Domain', 'Title', 'Description', 'Body', 'Link', 'timestamp',
       'Analyst_Average_Score', 'Analyst_Rank', 'Reference_Final_Score',
       'tokenized_Description', 'tokenized_Title', 'DS_stopwords_removed',
       'T_stopwords_removed', 'DS_porter_stemmed', 'T_porter_stemmed',
       'T_rejoined', 'DS_rejoined'],
      dtype='object')

In [47]:
df.to_csv('../data/preprocessed_news_data.csv')

You'll need your API key for this next cell. [Sign up to Cohere](https://os.cohere.ai/) and get one if you haven't yet.

In [ ]:
# Paste your API key here. Remember to not share publicly
api_key = '9Y0OqZQHpfb7vVrypsRXChE8VZzVAmZMCtmbIrhu'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [51]:
df[['Analyst_Average_Score','Analyst_Rank','Reference_Final_Score']]

,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
0,0.00,4,1.96
1,0.00,4,12.00
2,0.00,4,0.05
3,0.00,4,6.10
4,0.00,4,0.13
5,1.33,2,11.00
6,0.00,4,10.10
7,1.66,1,1.36
8,0.33,3,2.40
9,0.00,4,0.22


In [49]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


coherenewsExtractor = cohereExtractor([e[1] for e in news_examples], 
                                       [e[0] for e in news_examples], [],
                                       "", 
                                       "extract the news title from the post:")

# Uncomment to inspect the full prompt:
# print(coherenewsExtractor.make_prompt('<input text here>'))

NameError: name 'news_examples' is not defined

In [ ]:
# This is what the prompt looks like:
print(coherenewsExtractor.make_prompt('<input text here>'))

<input text here>
extract the news title from the post:


## Getting the data


In [ ]:
num_posts = 10

movies_list = get_post_titles(size=num_posts, 
      after=str(int(datetime.datetime(2021,1,1,0,0).timestamp())), 
      before=str(int(datetime.datetime(2022,1,1,0,0).timestamp())), 
      subreddit="movies", 
      sort_type="score", 
      sort="desc")

# Show the list
movies_list

## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [ ]:
results = []
for text in tqdm(movies_list):
    try:
        extracted_text = coherenewsExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


Let's look at the results:

In [ ]:
pd.DataFrame(data={'text': movies_list, 'extracted_text': results})

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

Let's look at some results:

In [ ]:
test_df.head()

Let's calculate the accuracy by comparing to the labeled examples

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

In [ ]:
test_df[test_df['correct']==0]

In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print(classification_report(test_df['label'].str.lower(), test_df['extracted_text'].str.lower()))